In [1]:
import sys

sys.path.append("../")
from hamilton_neural_network import (
    TrainTestData,
    LatentHamiltonianNeuralNetwork,
)
from hamilton_system import HamiltonianSystem
from pdf_models import NegLogIndepedentGaussians, NegLogThreeDimRosenbrock
import tensorflow as tf
import matplotlib.pyplot as plt
from no_u_turn.nuts import NoUTurnSampling

tf.random.set_seed(0)

2025-02-27 19:43:48.920544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740656629.007096    1137 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740656629.032438    1137 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 19:43:49.248587: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.18.0


In [2]:
U = NegLogThreeDimRosenbrock()
K = NegLogIndepedentGaussians(
    tf.constant([0.0, 0.0, 0.0]), tf.constant([1.0, 1.0, 1.0])
)
q0 = tf.constant([[0.0, 0.0, 0.0]])
p0 = tf.random.normal(q0.shape)
T = 100.0
leap_frog_per_unit = 80
num_samples = 40
num_train = int(0.9 * num_samples * leap_frog_per_unit * T)

I0000 00:00:1740656632.118060    1137 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4084 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:2b:00.0, compute capability: 7.5


In [3]:
train_test_data = TrainTestData(num_samples, T, leap_frog_per_unit, q0, p0, U=U, K=K)
samples = train_test_data()
tf.io.write_file("../exps/demo2_train_test_data.txt", tf.io.serialize_tensor(samples))

Generating samples...
Finished generating samples.


In [7]:
file = tf.io.read_file("../exps/demo2_train_test_data.txt")
train_test_data = tf.io.parse_tensor(file, out_type=tf.float32)
train_test_data = tf.random.shuffle(train_test_data)
train_data = train_test_data[:num_train, :]
test_data = train_test_data[num_train:, :]
lhnn = LatentHamiltonianNeuralNetwork(3, 64, 2)
lhnn.build(input_shape=(1, 6))
train_hist, test_hist = lhnn.train(
    5000,
    1000,
    4e-5,
    train_data,
    test_data,
    save_dir="../exps/demo2_lhnn.weights.h5",
    print_every=500,
)

Training started...
Epoch 0: Train loss 88.10517120361328, Test loss 92.82240295410156.


KeyboardInterrupt: 

In [ ]:
ax, fig = plt.subplots()
fig.plot(train_hist, label="train", color="red")
fig.plot(test_hist, label="test", color="blue")
fig.legend()
fig.set_yscale("log")
fig.set_xlabel("epochs (x500)")
fig.set_ylabel("loss")
plt.show()

In [ ]:
q0 = tf.constant([[0.0, 0.0, 0.0]])
nuts = NoUTurnSampling(
    num_samples=125000,
    q0=q0,
    dt=0.05,
    lhnn=lhnn,
    Hamiltonian=HamiltonianSystem(U=U, K=K),
    Delta_lf=1000.0,
    Delta_lhnn=10.0,
    num_lf_steps=20,
)
nuts(print_every=5000)
q_hist = tf.concat(nuts.q_hist, axis=0)
tf.io.write_file("../exps/demo2_q_hist.txt", tf.io.serialize_tensor(q_hist))
# plt.hist(q_hist.numpy()[5000:, 0].flatten(), bins=30, color="blue")
# plt.show()